Here we'll first be loading the FC matrices and explore their structure

In [1]:
#using read_dataset from Datasets/FC/create_dataset.py to read the dataset
from Datasets.FC.create_dataset import read_dataset_MDD

In [2]:
dataset = read_dataset_MDD()

In [3]:
#ok, let's explore the data a bit more
#dataset is a list object of torch_geometric.data objects

#let's see the first element
print(dataset[0])

#it's a dictionary object, let's see the keys
print(dataset[0].keys())

#ok, let's see the values
print(dataset[0].values())

#it has 4 keys, 'x', 'edge_index', 'edge_attr' and 'y' where y=0 menas the patient is healthy and y=1 means the patient has Autism Spectrum Disorder (ASD)
"""graph = Data(x=ROI.reshape(-1,116).float(),
                     edge_index=G.indices().reshape(2,-1).long(),
                     edge_attr=G.values().reshape(-1,1).float(),
                     y=y.long())"""

#how much data do we have?
len(dataset)


Data(x=[116, 116], edge_index=[2, 1294], edge_attr=[1294, 1], y=[1])
['edge_index', 'y', 'edge_attr', 'x']
ValuesView({'x': tensor([[ 0.0000,  0.2857,  0.0804,  ...,  0.2032,  0.1674,  0.0906],
        [ 0.2857,  0.0000, -0.3860,  ...,  0.1637, -0.0359,  0.1674],
        [ 0.0804, -0.3860,  0.0000,  ..., -0.0175, -0.0309,  0.0296],
        ...,
        [ 0.2032,  0.1637, -0.0175,  ...,  0.0000,  0.2329, -0.1452],
        [ 0.1674, -0.0359, -0.0309,  ...,  0.2329,  0.0000,  0.0183],
        [ 0.0906,  0.1674,  0.0296,  ..., -0.1452,  0.0183,  0.0000]]), 'edge_index': tensor([[  0,   0,   0,  ..., 113, 113, 114],
        [ 10,  12,  14,  ..., 111, 112, 108]]), 'edge_attr': tensor([[0.7785],
        [0.6966],
        [0.5463],
        ...,
        [0.6415],
        [0.4745],
        [0.5401]]), 'y': tensor([0])})


1604

In [4]:
#set the seed
import torch
torch.manual_seed(0)

MODEL = "GIN"
DATASET = "FC_MDD"

from models.models_FC import GIN_framework as framework # import the model

gnn = framework(dataset)

print(gnn.model)
print(gnn.train_idx)

MODELtri = "GINtri"
from models.models_FC import GIN_framework_tri as framework # import the model
gnntri = framework(dataset)

MODEL2 = "GIN2"
from models.models_FC import GIN_framework2 as framework2 # import the model
gnn2 = framework2(dataset)

MODEL3 = "GIN3"
from models.models_FC import GIN_framework3 as framework3 # import the model
gnn3 = framework3(dataset)

Net(
  (gin_layers): ModuleList(
    (0): GINConv(nn=Sequential(
      (0): Linear(in_features=116, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): BatchNorm(128)
    ))
    (1): GINConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): BatchNorm(128)
    ))
    (2): GINConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): BatchNorm(128)
    ))
    (3): GINConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): BatchNorm(128)
    ))
    (4): GINConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2)

/home/tpelletreaudur/.local/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/home/tpelletreaudur/.local/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/home/tpelletreaudur/.local/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/home/tpelletreaudur/.local/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [5]:
gnn.iterate()

Epoch: 020, Loss: 0.380, Test Loss: 1.026, Train Acc: 0.827 Test Acc: 0.506
Epoch: 040, Loss: 0.028, Test Loss: 1.371, Train Acc: 0.984 Test Acc: 0.617
Epoch: 060, Loss: 0.008, Test Loss: 1.690, Train Acc: 0.986 Test Acc: 0.654
Epoch: 080, Loss: 0.004, Test Loss: 1.835, Train Acc: 0.985 Test Acc: 0.642
Epoch: 100, Loss: 0.003, Test Loss: 1.950, Train Acc: 0.986 Test Acc: 0.630
Epoch: 120, Loss: 0.002, Test Loss: 2.048, Train Acc: 0.985 Test Acc: 0.617
Epoch: 140, Loss: 0.002, Test Loss: 2.131, Train Acc: 0.984 Test Acc: 0.617
Epoch: 160, Loss: 0.001, Test Loss: 2.202, Train Acc: 0.983 Test Acc: 0.605
Epoch: 180, Loss: 0.001, Test Loss: 2.257, Train Acc: 0.983 Test Acc: 0.593
Epoch: 200, Loss: 0.001, Test Loss: 2.309, Train Acc: 0.983 Test Acc: 0.593
Epoch: 220, Loss: 0.001, Test Loss: 2.358, Train Acc: 0.984 Test Acc: 0.593
Epoch: 240, Loss: 0.001, Test Loss: 2.394, Train Acc: 0.984 Test Acc: 0.593
Epoch: 260, Loss: 0.001, Test Loss: 2.430, Train Acc: 0.984 Test Acc: 0.593
Epoch: 280, 

In [6]:
gnntri.iterate()

Epoch: 020, Loss: 0.621, Test Loss: 0.788, Train Acc: 0.687 Test Acc: 0.506
Epoch: 040, Loss: 0.381, Test Loss: 0.940, Train Acc: 0.891 Test Acc: 0.556
Epoch: 060, Loss: 0.186, Test Loss: 1.308, Train Acc: 0.974 Test Acc: 0.543
Epoch: 080, Loss: 0.094, Test Loss: 1.283, Train Acc: 0.996 Test Acc: 0.568
Epoch: 100, Loss: 0.052, Test Loss: 1.363, Train Acc: 0.999 Test Acc: 0.580
Epoch: 120, Loss: 0.038, Test Loss: 1.512, Train Acc: 0.998 Test Acc: 0.593
Epoch: 140, Loss: 0.031, Test Loss: 1.496, Train Acc: 0.999 Test Acc: 0.605
Epoch: 160, Loss: 0.019, Test Loss: 1.561, Train Acc: 1.000 Test Acc: 0.605
Epoch: 180, Loss: 0.016, Test Loss: 1.593, Train Acc: 0.999 Test Acc: 0.605
Epoch: 200, Loss: 0.021, Test Loss: 1.630, Train Acc: 0.999 Test Acc: 0.667
Epoch: 220, Loss: 0.020, Test Loss: 1.723, Train Acc: 0.999 Test Acc: 0.593
Epoch: 240, Loss: 0.013, Test Loss: 1.719, Train Acc: 1.000 Test Acc: 0.593
Epoch: 260, Loss: 0.016, Test Loss: 1.746, Train Acc: 0.999 Test Acc: 0.630
Epoch: 280, 

In [9]:
# gnn2.iterate()

In [ ]:
# gnn3.iterate()

In [ ]:
# gnn3.cross_validate()

Fold: 1, Epoch: 50, Loss: 0.120, Train Acc: 0.961, Test Acc: 0.600
Fold: 1, Epoch: 100, Loss: 0.008, Train Acc: 1.000, Test Acc: 0.618


In [ ]:
#gnn.train()

In [7]:
# gnntri.save_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

Model saved in: models/FC_MDD_GINserver.pt


In [8]:
#save the model 
# gnn.save_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

# gnn2.save_model(path="models/"+DATASET+"_"+MODEL2+"server.pt")

# gnn3.save_model(path="models/"+DATASET+"_"+MODEL3+"server.pt")

Model saved in: models/FC_MDD_GINserver.pt


In [11]:
#load the model
gnn.load_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

# gnn2.load_model(path="models/"+DATASET+"_"+MODEL2+"server.pt")

# gnn3.load_model(path="models/"+DATASET+"_"+MODEL3+"server.pt")

# gnntri.load_model(path="models/"+DATASET+"_"+MODELtri+"server.pt")
#gnn3.save_model(path="models/"+DATASET+"_"+MODEL3+"server_iterate.pt")
#gnn3.load_model(path="models/"+DATASET+"_"+MODEL3+"server_iterate.pt")

In [12]:
gnn.evaluate()

Test Loss: 1.728, Train Acc: 0.970 Test Acc: 0.642


In [13]:
# gnntri.evaluate()

In [14]:
# gnn3.evaluate()

In [15]:
# from torch_geometric.data import DataLoader
# test_loader = DataLoader(dataset[gnn.test_idx], batch_size=1, shuffle=False)

# gnn3.evaluate2(test_loader)

In [16]:
train_features, test_features = gnn.evaluate_with_features2()

In [17]:
print(len(train_features[0]))
len(train_features), len(test_features)

9


(1523, 81)

In [18]:
import networkx as nx
def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Alternative metrics for disconnected graphs
        # Option 1: Use the average path length of the largest connected component
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        density = nx.density(G)
        avg_path_len = calculate_avg_path_length(G)
        num_cliques = len(list(nx.find_cliques(G)))
        num_triangles = sum(nx.triangles(G).values()) / 3
        num_squares = sum(nx.square_clustering(G).values()) / 4
        number_of_node_in_the_largest_fully_connected_component = len(max(nx.connected_components(G), key=len))
        #small_world = nx.algorithms.smallworld.sigma(G)

        properties.append((num_nodes, num_edges, density, avg_path_len, num_cliques, num_triangles, num_squares, number_of_node_in_the_largest_fully_connected_component)) #, small_world))
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties = compute_graph_properties(selected_dataset)

In [19]:
print(len(train_properties))
train_properties[0:5]



1523


[(116,
  488,
  0.07316341829085457,
  3.314842578710645,
  148,
  938.0,
  6.439699892510922,
  116),
 (114,
  598,
  0.0928427262847384,
  3.166744294364229,
  164,
  1378.0,
  7.519630514954544,
  114),
 (114,
  408,
  0.06334420121099209,
  3.391864617295451,
  135,
  511.0,
  5.279502013122494,
  114),
 (115,
  744,
  0.11350114416475973,
  2.796796338672769,
  198,
  1870.0,
  7.980704583203741,
  115),
 (112,
  706,
  0.11357786357786358,
  3.3745173745173744,
  181,
  2528.0,
  9.050100863174576,
  112)]

In [20]:
import pickle as pkl
#save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "wb") as f:
    pkl.dump(train_properties, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "wb") as f:
    pkl.dump(test_properties, f)

#load the properties
with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "rb") as f:
    train_properties = pkl.load(f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "rb") as f:
    test_properties = pkl.load(f)

In [22]:
"""
The embeddings of GIN are like this:
return F.log_softmax(x7, dim=-1), (x1, x2, x3, x4, x5, x_global, x6, x7, x8)
"""


import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
output_size = 1  # Predicting one property at a time
# Define the linear model
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Assume we have already evaluated to get features
train_features, test_features = gnn.evaluate_with_features2()

# Extract x embeddings
train_x = np.array([feat[0] for feat in train_features])
test_x = np.array([feat[0] for feat in test_features])

# Extract 2, 3, 4, global, 5, 6, 7 embeddings
train_x2 = np.array([feat[1] for feat in train_features])
test_x2 = np.array([feat[1] for feat in test_features])

train_x3 = np.array([feat[2] for feat in train_features])
test_x3 = np.array([feat[2] for feat in test_features])

train_x4 = np.array([feat[3] for feat in train_features])
test_x4 = np.array([feat[3] for feat in test_features])

train_x5 = np.array([feat[4] for feat in train_features])
test_x5 = np.array([feat[4] for feat in test_features])

train_x5 = np.array([feat[5] for feat in train_features])
test_x5 = np.array([feat[5] for feat in test_features])

train_x_global = np.array([feat[6] for feat in train_features])
test_x_global = np.array([feat[6] for feat in test_features])

train_x6 = np.array([feat[7] for feat in train_features])
test_x6 = np.array([feat[7] for feat in test_features])

train_x7 = np.array([feat[8] for feat in train_features])
test_x7 = np.array([feat[8] for feat in test_features])

# train_x8 = np.array([feat[9] for feat in train_features])
# test_x8 = np.array([feat[9] for feat in test_features])


# Compute graph properties
# train_properties = compute_graph_properties(gnn.dataset[gnn.train_idx])
# test_properties = compute_graph_properties(gnn.dataset[gnn.test_idx])

# Convert to PyTorch tensors
train_x = torch.tensor(train_x, dtype=torch.float32)
train_x2 = torch.tensor(train_x2, dtype=torch.float32)
train_x3 = torch.tensor(train_x3, dtype=torch.float32)
train_x4 = torch.tensor(train_x4, dtype=torch.float32)
train_x5 = torch.tensor(train_x5, dtype=torch.float32)
train_x_global = torch.tensor(train_x_global, dtype=torch.float32)
train_x6 = torch.tensor(train_x6, dtype=torch.float32)
train_x7 = torch.tensor(train_x7, dtype=torch.float32)
# train_x8 = torch.tensor(train_x8, dtype=torch.float32)

test_x = torch.tensor(test_x, dtype=torch.float32)
test_x2 = torch.tensor(test_x2, dtype=torch.float32)
test_x3 = torch.tensor(test_x3, dtype=torch.float32)
test_x4 = torch.tensor(test_x4, dtype=torch.float32)
test_x_global = torch.tensor(test_x_global, dtype=torch.float32)
test_x5 = torch.tensor(test_x5, dtype=torch.float32)
test_x6 = torch.tensor(test_x6, dtype=torch.float32)
test_x7 = torch.tensor(test_x7, dtype=torch.float32)
# test_x8 = torch.tensor(test_x8, dtype=torch.float32)

train_y = torch.tensor(train_properties, dtype=torch.float32)
test_y = torch.tensor(test_properties, dtype=torch.float32)

# Train and evaluate a model for each graph property and each embedding
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings = [(train_x, test_x), (train_x2, test_x2), (train_x3, test_x3), (train_x4, test_x4), (train_x5, test_x5), (train_x_global, test_x_global), (train_x6, test_x6), (train_x7, test_x7)]#, (train_x8, test_x8)]
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global',  'x6', 'x7']#, 'x8']

In [ ]:
#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y[:, i].cpu().numpy()
            test_target = test_y[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "wb") as f:
    pkl.dump(results, f)

Epoch [1000/800000], Property: num_nodes, Loss: 5852.8057
Epoch [2000/800000], Property: num_nodes, Loss: 3221.9648
Epoch [3000/800000], Property: num_nodes, Loss: 2061.8516
Epoch [4000/800000], Property: num_nodes, Loss: 1311.6721
Epoch [5000/800000], Property: num_nodes, Loss: 808.2582
Epoch [6000/800000], Property: num_nodes, Loss: 494.6854
Epoch [7000/800000], Property: num_nodes, Loss: 327.9342
Epoch [8000/800000], Property: num_nodes, Loss: 250.8599
Epoch [9000/800000], Property: num_nodes, Loss: 211.0283
Epoch [10000/800000], Property: num_nodes, Loss: 182.4039
Epoch [11000/800000], Property: num_nodes, Loss: 158.2924
Epoch [12000/800000], Property: num_nodes, Loss: 138.0651
Epoch [13000/800000], Property: num_nodes, Loss: 121.8526
Epoch [14000/800000], Property: num_nodes, Loss: 109.1967
Epoch [15000/800000], Property: num_nodes, Loss: 99.4661
Epoch [16000/800000], Property: num_nodes, Loss: 92.0923
Epoch [17000/800000], Property: num_nodes, Loss: 86.4896
Epoch [18000/800000], 

Epoch [12000/800000], Property: num_edges, Loss: 51115.0312
Epoch [13000/800000], Property: num_edges, Loss: 45694.0898
Epoch [14000/800000], Property: num_edges, Loss: 40906.9727
Epoch [15000/800000], Property: num_edges, Loss: 36672.3711
Epoch [16000/800000], Property: num_edges, Loss: 32951.0977
Epoch [17000/800000], Property: num_edges, Loss: 29725.1055
Epoch [18000/800000], Property: num_edges, Loss: 26974.8750
Epoch [19000/800000], Property: num_edges, Loss: 24674.1699
Epoch [20000/800000], Property: num_edges, Loss: 22788.0293
Epoch [21000/800000], Property: num_edges, Loss: 21276.9141
Epoch [22000/800000], Property: num_edges, Loss: 20089.8301
Epoch [23000/800000], Property: num_edges, Loss: 19172.9082
Epoch [24000/800000], Property: num_edges, Loss: 18473.6582
Epoch [25000/800000], Property: num_edges, Loss: 17942.0957
Epoch [26000/800000], Property: num_edges, Loss: 17529.6152
Epoch [27000/800000], Property: num_edges, Loss: 17196.3750
Epoch [28000/800000], Property: num_edge

Epoch [148000/800000], Property: num_edges, Loss: 13149.0674
Epoch [149000/800000], Property: num_edges, Loss: 13144.5947
Epoch [150000/800000], Property: num_edges, Loss: 13140.1357
Epoch [151000/800000], Property: num_edges, Loss: 13135.6924
Epoch [152000/800000], Property: num_edges, Loss: 13131.2578
Epoch [153000/800000], Property: num_edges, Loss: 13126.8418
Epoch [154000/800000], Property: num_edges, Loss: 13122.4375
Epoch [155000/800000], Property: num_edges, Loss: 13118.0459
Epoch [156000/800000], Property: num_edges, Loss: 13113.6689
Epoch [157000/800000], Property: num_edges, Loss: 13109.3076
Epoch [158000/800000], Property: num_edges, Loss: 13104.9570
Epoch [159000/800000], Property: num_edges, Loss: 13100.6240
Epoch [160000/800000], Property: num_edges, Loss: 13096.2998
Epoch [161000/800000], Property: num_edges, Loss: 13091.9922
Epoch [162000/800000], Property: num_edges, Loss: 13087.6982
Epoch [163000/800000], Property: num_edges, Loss: 13083.4170
Epoch [164000/800000], P

Epoch [283000/800000], Property: num_edges, Loss: 12668.8711
Epoch [284000/800000], Property: num_edges, Loss: 12666.2178
Epoch [285000/800000], Property: num_edges, Loss: 12663.5840
Epoch [286000/800000], Property: num_edges, Loss: 12660.9590
Epoch [287000/800000], Property: num_edges, Loss: 12658.3516
Epoch [288000/800000], Property: num_edges, Loss: 12655.7568
Epoch [289000/800000], Property: num_edges, Loss: 12653.1768
Epoch [290000/800000], Property: num_edges, Loss: 12650.6104
Epoch [291000/800000], Property: num_edges, Loss: 12648.0596
Epoch [292000/800000], Property: num_edges, Loss: 12645.5205
Epoch [293000/800000], Property: num_edges, Loss: 12642.9990
Epoch [294000/800000], Property: num_edges, Loss: 12640.4912
Epoch [295000/800000], Property: num_edges, Loss: 12637.9961
Epoch [296000/800000], Property: num_edges, Loss: 12635.5137
Epoch [297000/800000], Property: num_edges, Loss: 12633.0479
Epoch [298000/800000], Property: num_edges, Loss: 12630.5947
Epoch [299000/800000], P

Epoch [418000/800000], Property: num_edges, Loss: 12440.8672
Epoch [419000/800000], Property: num_edges, Loss: 12440.1250
Epoch [420000/800000], Property: num_edges, Loss: 12439.3984
Epoch [421000/800000], Property: num_edges, Loss: 12438.6846
Epoch [422000/800000], Property: num_edges, Loss: 12437.9814
Epoch [423000/800000], Property: num_edges, Loss: 12437.2939
Epoch [424000/800000], Property: num_edges, Loss: 12436.6201
Epoch [425000/800000], Property: num_edges, Loss: 12435.9580
Epoch [426000/800000], Property: num_edges, Loss: 12435.3105
Epoch [427000/800000], Property: num_edges, Loss: 12434.6748
Epoch [428000/800000], Property: num_edges, Loss: 12434.0537
Epoch [429000/800000], Property: num_edges, Loss: 12433.4463
Epoch [430000/800000], Property: num_edges, Loss: 12432.8506
Epoch [431000/800000], Property: num_edges, Loss: 12432.2695
Epoch [432000/800000], Property: num_edges, Loss: 12431.7021
Epoch [433000/800000], Property: num_edges, Loss: 12431.1475
Epoch [434000/800000], P

...

Epoch [16000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 128.5778
Epoch [17000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 114.7506
Epoch [18000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 103.1693
Epoch [19000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 93.5649
Epoch [20000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 85.6485
Epoch [21000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 79.1698
Epoch [22000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 73.9353
Epoch [23000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 69.7341
Epoch [24000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 66.3669
Epoch [25000/800000], Property: number_of_nodes_in_the_larges

Epoch [95000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 14.1445
Epoch [96000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 13.8873
Epoch [97000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 13.6411
Epoch [98000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 13.4069
Epoch [99000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 13.1831
Epoch [100000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.9698
Epoch [101000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.7677
Epoch [102000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.5768
Epoch [103000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.3961
Epoch [104000/800000], Property: number_of_nodes_in_the_larg

Epoch [77000/800000], Property: num_nodes, Loss: 12.7283
Epoch [78000/800000], Property: num_nodes, Loss: 12.3976
Epoch [79000/800000], Property: num_nodes, Loss: 12.0741
Epoch [80000/800000], Property: num_nodes, Loss: 11.7579
Epoch [81000/800000], Property: num_nodes, Loss: 11.4491
Epoch [82000/800000], Property: num_nodes, Loss: 11.1475
Epoch [83000/800000], Property: num_nodes, Loss: 10.8535
Epoch [84000/800000], Property: num_nodes, Loss: 10.5683
Epoch [85000/800000], Property: num_nodes, Loss: 10.2907
Epoch [86000/800000], Property: num_nodes, Loss: 10.0203
Epoch [87000/800000], Property: num_nodes, Loss: 9.7573
Epoch [88000/800000], Property: num_nodes, Loss: 9.5015
Epoch [89000/800000], Property: num_nodes, Loss: 9.2528
Epoch [90000/800000], Property: num_nodes, Loss: 9.0115
Epoch [91000/800000], Property: num_nodes, Loss: 8.7774
Epoch [92000/800000], Property: num_nodes, Loss: 8.5511
Epoch [93000/800000], Property: num_nodes, Loss: 8.3330
Epoch [94000/800000], Property: num_no

Epoch [34000/800000], Property: num_edges, Loss: 25608.7031
Epoch [35000/800000], Property: num_edges, Loss: 24418.0547
Epoch [36000/800000], Property: num_edges, Loss: 23412.0059
Epoch [37000/800000], Property: num_edges, Loss: 22576.7676
Epoch [38000/800000], Property: num_edges, Loss: 21896.9238
Epoch [39000/800000], Property: num_edges, Loss: 21352.2129
Epoch [40000/800000], Property: num_edges, Loss: 20914.7148
Epoch [41000/800000], Property: num_edges, Loss: 20551.7109
Epoch [42000/800000], Property: num_edges, Loss: 20232.8984
Epoch [43000/800000], Property: num_edges, Loss: 19937.7695
Epoch [44000/800000], Property: num_edges, Loss: 19657.8008
Epoch [45000/800000], Property: num_edges, Loss: 19390.9863
Epoch [46000/800000], Property: num_edges, Loss: 19137.2051
Epoch [47000/800000], Property: num_edges, Loss: 18896.7266
Epoch [48000/800000], Property: num_edges, Loss: 18669.0371
Epoch [49000/800000], Property: num_edges, Loss: 18453.5605
Epoch [50000/800000], Property: num_edge

Epoch [170000/800000], Property: num_edges, Loss: 13268.2021
Epoch [171000/800000], Property: num_edges, Loss: 13259.2568
Epoch [172000/800000], Property: num_edges, Loss: 13250.4062
Epoch [173000/800000], Property: num_edges, Loss: 13241.6455
Epoch [174000/800000], Property: num_edges, Loss: 13232.9756
Epoch [175000/800000], Property: num_edges, Loss: 13224.3926
Epoch [176000/800000], Property: num_edges, Loss: 13215.8975
Epoch [177000/800000], Property: num_edges, Loss: 13207.4854
Epoch [178000/800000], Property: num_edges, Loss: 13199.1572
Epoch [179000/800000], Property: num_edges, Loss: 13190.9189
Epoch [180000/800000], Property: num_edges, Loss: 13182.7617
Epoch [181000/800000], Property: num_edges, Loss: 13174.6904
Epoch [182000/800000], Property: num_edges, Loss: 13166.7051
Epoch [183000/800000], Property: num_edges, Loss: 13158.8037
Epoch [184000/800000], Property: num_edges, Loss: 13150.9873
Epoch [185000/800000], Property: num_edges, Loss: 13143.2539
Epoch [186000/800000], P

In [ ]:
#load results
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "rb") as f:
    results = pkl.load(f)

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7', 'x8']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', 'orange']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_name)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig("results/"+DATASET+"_"+MODEL+"test_R2_plot.png")


In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7', 'x8']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', 'orange']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_name)][2]
        if train_r2 < -0.05:
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)

    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig("results/"+DATASET+"_"+MODEL+"train_R2_plot.png")

#### Test with more properties

In [ ]:
import networkx as nx
import numpy as np

def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Use the average path length of the largest connected component for disconnected graphs
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    
def betweenness_centralization(G):
    n = len(G)
    betweenness = nx.betweenness_centrality(G)
    max_betweenness = max(betweenness.values())
    centralization = sum(max_betweenness - bet for bet in betweenness.values())
    if n > 2:
        centralization /= (n - 1) * (n - 2) / 2
    return centralization

def pagerank_centralization(G, alpha=0.85):
    n = len(G)
    pagerank = nx.pagerank(G, alpha=alpha)
    max_pagerank = max(pagerank.values())
    centralization = sum(max_pagerank - pr for pr in pagerank.values())
    if n > 1:
        centralization /= (n - 1)
    return centralization

def clustering_properties(G):
    average_clustering = nx.average_clustering(G)
    transitivity = nx.transitivity(G)
    return average_clustering, transitivity

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        
        # Number of nodes
        num_nodes = G.number_of_nodes()
        
        # Number of edges
        num_edges = G.number_of_edges()
        
        # Density
        density = nx.density(G)
        
        # Average Path Length
        avg_path_len = calculate_avg_path_length(G)
        
        # Diameter
        if nx.is_connected(G):
            diameter = nx.diameter(G)
        else:
            # Use the diameter of the largest connected component for disconnected graphs
            components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
            largest_component = max(components, key=len)
            diameter = nx.diameter(largest_component)
        
        # Radius
        if nx.is_connected(G):
            radius = nx.radius(G)
        else:
            radius = nx.radius(largest_component)
        
        # Clustering Coefficient
        clustering_coeff = nx.average_clustering(G)
        
        # Transitivity
        transitivity = nx.transitivity(G)
        
        # Assortativity
        assortativity = nx.degree_assortativity_coefficient(G)
        
        # Number of Cliques
        num_cliques = len(list(nx.find_cliques(G)))
        
        # Number of Triangles
        num_triangles = sum(nx.triangles(G).values()) / 3
        
        # Number of Squares (4-cycles)
        num_squares = sum(nx.square_clustering(G).values()) / 4
        
        # Size of the Largest Connected Component
        largest_component_size = len(max(nx.connected_components(G), key=len))
        
        # Average Degree
        degrees = [d for n, d in G.degree()]
        avg_degree = np.mean(degrees)
        
        # Betweenness Centrality
        betweenness_centrality = nx.betweenness_centrality(G)
        avg_betweenness_centrality = np.mean(list(betweenness_centrality.values()))
        
        # Eigenvalues of the Adjacency Matrix (for spectral properties)
        eigenvalues = np.linalg.eigvals(nx.adjacency_matrix(G).todense())
        spectral_radius = max(eigenvalues)
        algebraic_connectivity = sorted(eigenvalues)[1]  # second smallest eigenvalue
        
        # Graph Laplacian Eigenvalues
        laplacian_eigenvalues = np.linalg.eigvals(nx.laplacian_matrix(G).todense())
        graph_energy = sum(abs(laplacian_eigenvalues))
        
        # Small-World-ness
        # Compare clustering coefficient and average path length with those of a random graph
        random_graph = nx.gnm_random_graph(num_nodes, num_edges)
        random_clustering_coeff = nx.average_clustering(random_graph)
        random_avg_path_len = calculate_avg_path_length(random_graph)
        small_world_coefficient = (clustering_coeff / random_clustering_coeff) / (avg_path_len / random_avg_path_len)

        # Calculate Betweenness Centralization
        betweenness_cent = betweenness_centralization(G)
        print(f"Betweenness Centralization: {betweenness_cent}")

        # Calculate PageRank Centralization
        pagerank_cent = pagerank_centralization(G)
        print(f"PageRank Centralization: {pagerank_cent}")

        # Calculate Clustering properties
        avg_clustering, transitivity = clustering_properties(G)
        print(f"Average Clustering Coefficient: {avg_clustering}")
        print(f"Transitivity: {transitivity}")
        
        properties.append((
            num_nodes,
            num_edges,
            density,
            avg_path_len,
            diameter,
            radius,
            clustering_coeff,
            transitivity,
            assortativity,
            num_cliques,
            num_triangles,
            num_squares,
            largest_component_size,
            avg_degree,
            avg_betweenness_centrality,
            spectral_radius,
            algebraic_connectivity,
            graph_energy,
            small_world_coefficient
        ))
    return properties


train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties_long = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties_long = compute_graph_properties(selected_dataset)

#save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "wb") as f:
    pkl.dump(train_properties_long, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "wb") as f:
    pkl.dump(test_properties_long, f)



In [ ]:
#load the properties
# with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "rb") as f:
#     train_properties_long = pkl.load(f)

# with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "rb") as f:
#     test_properties_long = pkl.load(f)

In [ ]:
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']
train_y_long = torch.tensor(train_properties_long, dtype=torch.float32)
test_y_long = torch.tensor(test_properties_long, dtype=torch.float32)
#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names_long):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y_long[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y_long[:, i].cpu().numpy()
            test_target = test_y_long[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv_long.pkl", "wb") as f:
    pkl.dump(results, f)

In [ ]:
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']

colors_long = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

plt.figure(figsize=(12, 8))

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_names_long)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+'_'+MODEL+'test_R2_plot_long.png')

In [ ]:
plt.figure(figsize=(12, 8))
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_names_long)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)

    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+'_'+MODEL+'train_R2_plot_long.png')
    